# 02. DNN with 텐서플로우케라스
---

본 내용은 [딥 러닝을 이용한 자연어 처리 입문](https://wikidocs.net/book/2155) 을 바탕으로 공부하고 정리한 것임을 밝힙니다.

**목차**
>* 순전파
  1. 행렬곱 연산
  2. 병렬 연산
* 다층 퍼셉트론의 순전파
  1. 행렬곱 연산
* 손실함수
  1. MSE(Mean Squared Error, MSE)
  2. 이진 크로스 엔트로피(Binary Cross-Entropy)
  3. 카테고리칼 크로스 엔트로피(Categorical Cross-Entropy)
  4. 그 외에 다양한 손실 함수들
* 옵티마이저
  1. 모멘텀(Momentum)
  2. 아다그라드(Adagrad)
  3. 알엠에스프롭(RMSprop)
  4. 아담(Adam)
* 역전파

텐서플로우 케라스 : https://www.tensorflow.org/?hl=ko

---
## 순전파(Foward Propagation)
---
활성화 함수, 은닉층의 수, 각 은닉층의 뉴런 수 등 딥 러닝 모델을 설계하고나면 입력값은 입력층, 은닉층을 지나면서 각 층에서의 가중치와 함께 연산되며 출력층으로 향합니다. 그리고 출력층에서 모든 연산을 마친 예측값이 나오게 됩니다. 이와 같이 입력층에서 출력층 방향으로 예측값의 연산이 진행되는 과정을 순전파라고 합니다.

<p align='center'>
<img src=https://wikidocs.net/images/page/37406/nn2_final_final.PNG width=400>






### **(1) 행렬곱 연산**

아래 신경망 그림에서 화살표 각각은 가중치 를 의미하고 있습니다. 3개의 뉴런과 2개의 뉴런 사이에는 총 6개의 화살표가 존재하는데, 이는 위 신경망에서 가중치 의 개수가 6개임을 의미합니다.


<p align='center'>
<img src=https://wikidocs.net/images/page/150781/nn.PNG width=450>
</p>


일반적으로 동그란 뉴런과 화살표로 표현하는 인공 신경망의 그림에서는 편향 의 경우에는 편의상 생략되는 경우가 많지만, 인공 신경망 내부적으로는 편향 의 연산 또한 존재합니다. 위 그림에서 뉴런과 화살표로 표현한 인공 신경망의 그림에서는 편향을 표현하지 않았지만, 행렬 연산식에서는  표현하였습니다. 


**텐서플로우 를 이용한 구현**

In [1]:
from tensorflow.keras import models, layers

In [ ]:
x = layers.Input(shape=(3,))
y = layers.Dense(2)(x)

model = models.Model(x, y)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3)]               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 8         
                                                                 
Total params: 8
Trainable params: 8
Non-trainable params: 0
_________________________________________________________________


<p align='center'>
<img src=https://wikidocs.net/images/page/150781/matrix_multiplication.PNG width=450>
</p>


* 학습 파라미터 확인
  * `get_weights()`

In [ ]:
W, B = model.get_weights()
print(W, W.shape)
print(B, B.shape)

[[ 1.0068762  -1.0460497 ]
 [-1.0541595  -0.74557555]
 [-0.77009255  0.78064084]] (3, 2)
[0. 0.] (2,)


### **(2) 병렬연산**

인공 신경망을 행렬곱으로 구현할 때의 흥미로운 점은 행렬곱을 사용하면 병렬 연산도 가능하다는 점입니다. 인공 신경망이 4개의 샘플을 동시에 처리해본다고 가정해봅시다. 4개의 샘플을 하나의 행렬 로 정의하고 인공 신경망의 순전파를 행렬곱으로 표현하면 다음과 같습니다.

<p align='center'>
<img src=https://wikidocs.net/images/page/150781/parallel_nn.PNG width=500>
</p>

여기서 혼동하지 말아야 할 것은 인공 신경망의 4개의 샘플을 동시에 처리하고 있지만, 여기서 학습가능한 매개변수의 수는 여전히 8개라는 점입니다. 이렇게 인공 신경망이 다수의 샘플을 동시에 처리하는 것을 우리는 **'배치 연산'** 이라고 부릅니다.

---
## 다층 퍼셉트론의 순전파
---

<p align='center'>
<img src=https://wikidocs.net/images/page/24987/neuralnetwork_final.PNG width=320>

### **(1) 행렬곱 연산**

위 그림에 해당하는 인공신경망의 데이터와 학습 파라미터의 크기를 구하세요


* $x, z_1, z_2, y$


In [ ]:
'''
x  : 4
z1 : 8
z2 : 8
y  : 3 
'''

* $W_1, W_2, W_3$ 


In [ ]:
'''
W1 : (4,8)
W2 : (8,8)
W3 : (8,3)
'''

* $b_1, b_2, b_3$ 


In [ ]:
'''
b1 = (8)
b2 = (8)
b3 = (3)
'''

**텐서플로우 를 이용한 구현**

In [5]:
x = layers.Input(shape=4)
z1 = layers.Dense(8)(x)
z2 = layers.Dense(8)(z1)
y = layers.Dense(3)(z2)

model = models.Model(x, y)

In [6]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense_3 (Dense)             (None, 8)                 40        
                                                                 
 dense_4 (Dense)             (None, 8)                 72        
                                                                 
 dense_5 (Dense)             (None, 3)                 27        
                                                                 
Total params: 139
Trainable params: 139
Non-trainable params: 0
_________________________________________________________________



* 학습 파라미터 확인
  * `get_weights()`

In [7]:
# w1, b1, w2, b2, w3, b3
for param in model.get_weights():
  print(param.shape)

(4, 8)
(8,)
(8, 8)
(8,)
(8, 3)
(3,)


---
## 손실함수 (Loss function)
---

**손실함수란?**

> 예측 값과 실제 값의 차이를 계산하는 함수(식)


<p align='center'>
<img src=https://wikidocs.net/images/page/36033/%EC%86%90%EC%8B%A4%ED%95%A8%EC%88%98.PNG
 width=450>

### **(1)  MSE(Mean Squared Error, MSE)**
---
평균 제곱 오차는 선형 회귀를 학습할 때 배웠던 손실 함수입니다. 연속형 변수를 예측할 때 사용됩니다.

**텐서플로우 를 이용한 구현**

* `mse` model.compile(loss='mse', optimizer='sgd')
* `tf.keras.losses.MeanSquaredError()`
(import tensorflow as tf, 
from tensorflow.keras import losses)

In [ ]:
import tensorflow as tf

model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer='sgd')

### **(2) 이진 크로스 엔트로피(Binary Cross-Entropy)**
---

이항 교차 엔트로피라고도 부르는 손실 함수입니다. 출력층에서 시그모이드 함수를 사용하는 이진 분류 (Binary Classification)의 경우 `binary_crossentropy`를 사용합니다.



**텐서플로우 를 이용한 구현**

* `binary_crossentropy`
* `tf.keras.losses.BinaryCrossentropy()`

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='sgd')

### **(3) 카테고리칼 크로스 엔트로피(Categorical Cross-Entropy)**
---

범주형 교차 엔트로피라고도 부르는 손실 함수입니다. 출력층에서 소프트맥스 함수를 사용하는 다중 클래스 분류(Multi-Class Classification)일 경우 `categorical_crossentropy`를 사용합니다. 

**텐서플로우 를 이용한 구현**

* `categorical_crossentropy`
* `tf.keras.losses.CategoricalCrossentropy()`

원-핫 인코딩 과정을 생략하고 정수값 가진 레이블에 대해서 학습을 수행할 때는 아래 함수를 사용할 수 있습니다.

* `sparse_categorical_crossentropy`
* `tf.keras.losses.SparseCategoricalCrossentropy()`

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd')

### **(4) 그 외에 다양한 손실 함수들**
---
아래의 텐서플로우 공식 문서 링크에서 방금 언급하지 않은 손실 함수 외에도 다양한 손실 함수들을 확인할 수 있습니다.

https://www.tensorflow.org/api_docs/python/tf/keras/losses

---
## 옵티마이저 (Optimizer)
---

**옵티마이저란?**

> 학습 파라미터를 조절해서 오차를 줄여가도록 하는 함수


### **(1) 모멘텀(Momentum)**
---
전체 함수에 걸쳐 최소값을 **글로벌 미니멈**(Global Minimum) 이라고 하고, 글로벌 미니멈이 아닌 특정 구역에서의 최소값인 **로컬 미니멈**(Local Minimum) 이라고 합니다. 


<p align='center'>
<img src=https://wikidocs.net/images/page/24987/%EB%A1%9C%EC%BB%AC%EB%AF%B8%EB%8B%88%EB%A9%88.PNG
 width=250>


![]()

**모멘텀(Momentum)**은 관성이라는 물리학의 법칙을 응용한 방법입니다. 모멘텀은 **경사 하강법에 관성을 더 해줍니다.** 모멘텀은 경사 하강법에서 계산된 접선의 기울기에 한 시점(step) 전의 접선의 기울기값을 일정한 비율만큼 반영합니다. 이렇게 하면 마치 언덕에서 공이 내려올 때, 중간에 작은 웅덩이에 빠지더라도 관성의 힘으로 넘어서는 효과를 줄 수 있습니다.

**텐서플로우 를 이용한 구현**

* `sgd`
* `tf.keras.optimizers.SGD(lr=0.01, momentum=0.9)`


In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(lr=0.01, momentum=0.9))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


### **(2) 아다그라드(Adagrad)**
---

매개변수들은 각자 의미하는 바가 다른데, 모든 매개변수에 동일한 학습률(learning rate)을 적용하는 것은 비효율적입니다. **아다그라드**는 각 매개변수에 **서로 다른 학습률을 적용시킵니다.** 이때 변화가 많은 매개변수는 학습률이 작게 설정되고 변화가 적은 매개변수는 학습률을 높게 설정시킵니다. 


**텐서플로우 를 이용한 구현**

* `adagrad`
* `tf.keras.optimizers.Adagrad(lr=0.01, epsilon=1e-6)
`


In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adagrad')

### **(3) 알엠에스프롭(RMSprop)**
---

아다그라드는 나중에 가서는 학습률이 지나치게 떨어진다는 단점이 있는데 이를 다른 수식으로 대체하여 이러한 단점을 개선하였습니다.


**텐서플로우 를 이용한 구현**

* `rmsprop`
* `tf.keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-06)`


In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop')

### **(4) 아담(Adam)**
---

아담은 알엠에스프롭과 모멘텀 두 가지를 합친 듯한 방법으로, 방향과 학습률 두 가지를 모두 잡기 위한 방법입니다.




**텐서플로우 를 이용한 구현**

* `adam`
* `tf.keras.optimizers.Adam(lr=0.001)`


In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

---
## 역전파 (Back propagation)
---

옵티마이저가 수행하는 역할이 바로 역전파입니다. 순전파와 역전파가 한번 진행 될때마다 역전파 알고리즘은 모델의 학습 파라미터에 대한 오차의 미분을 계산합니다. 오차를 감소시키기 위해 각 가중치와 편향이 어떻게 바뀌어야 할 지 알 수 있으며 이 계산과정을 역전파라 합니다. 


<p align='center'>
<img src=https://wikidocs.net/images/page/37406/nn4.PNG
 width=400>
